In [1]:
!pip install mrjob

In [2]:
#importing pandas libary
import pandas as pd

In [3]:
#reading csv file
data=pd.read_csv('yelp.csv')
#displaying first 5 rows
data.head(5)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [4]:
#getting number of rows and columns in a data frame
data.shape

(10000, 10)

In [5]:
#printing all columns in data frame
data.columns

Index(['business_id', 'date', 'review_id', 'stars', 'text', 'type', 'user_id',
       'cool', 'useful', 'funny'],
      dtype='object')

In [6]:
#replacing newline characters with a space in all columns with a regular expression
data = data.replace(r'\n',' ', regex=True)
#saving modified data frame to new csv file
data.to_csv("yelp_df.csv")

# (1) AVERAGE NUMBER OF WORDS IN EACH REVIEW

In [7]:
%%file Avgcount2.py
from mrjob.job import MRJob
import csv

class AvgWordCount(MRJob):
# define a MapReduce class named AvgWordCount, inheriting from MRJob

    def mapper(self, _, line):
        #create a csv reader and converts the line to list
        id,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny=next(csv.reader([line]))
        
        # emit key-value pair where the key is 0 and the value is the length of the text split by spaces

        yield 0,len(text.split(" "))
        
    # reducer function, called for each unique key    
    def reducer(self, cell, values):
        # convert the values iterator to a list
        values = list(values)
        # calculate the sum and count of words
        total_words = sum(values)
        word_count = len(values)
        
        # calculate the average number of words
        average_words = total_words / word_count
        yield None, average_words


# entry point to run the MapReduce job if this script is executed

if __name__ == '__main__':
    AvgWordCount.run()

    


Writing Avgcount2.py


In [8]:
# importing AvgWordCount class from Avgcount2 module
import Avgcount2

# create an instance of AvgWordCount MapReduce job 

mr_job = Avgcount2.AvgWordCount(args=['yelp_df.csv'])
# using make_runner method to create a runner for the MapReduce job

with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        # print each key 
        print(key,value)

No configs specified for inline runner


None 136.32086791320867


# (2) COUNT OF REVIEWS BY YEAR-MONTH

In [9]:
%%file yearmonth.py
from mrjob.job import MRJob
import csv

# define a MapReduce job class named MRyearmonth, inheriting from MRJob
class MRyearmonth(MRJob):
    
    def mapper(self, _,line):
        # split CSV line into individual values
        
        id,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny=next(csv.reader([line]))
        
        #extract the year and month from the date column and emit a key-value pair
        yield date[:7],1
        
    #define reducer function
    def reducer(self,key,values):
        #sum the values for each key
        yield key,sum(values) 
        
if __name__=='__main__':
    MRyearmonth.run()

Writing yearmonth.py


In [10]:
import yearmonth

# create an instance of  MRyearmonth MapReduce job 
mr_job = yearmonth.MRyearmonth(args=['yelp_df.csv'])

# use a context manager to manage the runner for the MapReduce job

with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(key,value)

No configs specified for inline runner


2005-04 1
2005-07 2
2005-12 1
2006-01 6
2006-02 9
2006-04 2
2006-05 1
2006-06 5
2006-07 2
2006-08 9
2006-09 4
2006-10 5
2006-11 6
2006-12 6
2007-01 14
2007-02 20
2007-03 42
2007-04 8
2007-05 23
2007-06 12
2007-07 35
2007-08 29
2007-09 26
2007-10 23
2007-11 28
2007-12 25
2008-01 46
2008-02 48
2008-03 47
2008-04 53
2008-05 65
2008-06 76
2008-07 80
2008-08 75
2008-09 59
2008-10 79
2008-11 66
2008-12 71
2009-01 108
2009-02 79
2009-03 126
2009-04 101
2009-05 101
2009-06 67
2009-07 95
2009-08 98
2009-09 113
2009-10 101
2009-11 78
2009-12 104
2010-01 154
2010-02 148
2010-03 168
2010-04 148
2010-05 154
2010-06 118
2010-07 160
2010-08 201
2010-09 150
2010-10 144
2010-11 147
2010-12 160
2011-01 239
2011-02 216
2011-03 263
2011-04 263
2011-05 229
2011-06 230
2011-07 236
2011-08 266
2011-09 193
2011-10 204
2011-11 203
2011-12 249
2012-01 304
2012-02 219
2012-03 259
2012-04 265
2012-05 275
2012-06 272
2012-07 281
2012-08 249
2012-09 239
2012-10 258
2012-11 208
2012-12 196
2013-01 52
date 1


# (3) AVERAGE RATING OF ANY REVIEW MARKED "COOL"

In [11]:
%%file avgrating.py
#importing necessary modules
from mrjob.job import MRJob
import csv
from statistics import mean

# define the MRavgrating class, inheriting from MRJob

class MRavgrating(MRJob):
    
    # mapper function processes each line of input
    def mapper(self, _, line):
        # parse the CSV row from the input line
        row = next(csv.reader([line]))
        id,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny=row
        
        # Check conditions to filter relevant data
        if (cool!=0 and stars != "stars"):
            
            # emit a key-value pair with key=1 and value=stars as an integer
            yield 1,int(stars)
            
    # reducer function calculates the average of received values    
    def reducer(self, key, values):
        values = list(values)
        a=sum(values)
        b=len(values)
        average_words = a / b
        yield None, average_words

# entry point to run the MRavgrating MapReduce job
if __name__ == '__main__':
    MRavgrating.run()

Writing avgrating.py


In [12]:
import avgrating
mr_job = avgrating.MRavgrating(args=['yelp_df.csv'])
with mr_job.make_runner() as runner:
    runner.run()
    
    # Iterate through the key-value pairs in the output and print the average stars
    for key, value in mr_job.parse_output(runner.cat_output()):
        print("average",value)

No configs specified for inline runner


average 3.7775
